In [1]:
%pip install -q torch diffusers transformers pillow matplotlib ipywidgets accelerate hf_xet


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from compute_steering_vectors import compute_steering_vectors

model_name = 'sd21'

steering_vectors_cow = compute_steering_vectors(
    model_name=model_name,
    mode='concrete',
    concept_pos='a cow',
    num_denoising_steps=50,
    max_prompts=8,
)

steering_vectors_pig = compute_steering_vectors(
    model_name=model_name,
    mode='concrete',
    concept_pos='a pig',
    num_denoising_steps=50,
    max_prompts=8,
)

Вычисляем управляющие векторы: модель=sd21, режим=concrete, концепция=a cow


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Обрабатываем 8 пар промптов...


  0%|          | 0/8 [00:00<?, ?it/s]

Промпт 1/8: 'cyberpunk with a cow' и 'cyberpunk'
down 6
up 15
mid 16


...

  0%|          | 0/50 [00:00<?, ?it/s]

Промпт 6/8: 'hyperrealistic with a cow' и 'hyperrealistic'
down 6
up 15
mid 16


  0%|          | 0/50 [00:00<?, ?it/s]

down 6
up 15
mid 16


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
from collections import defaultdict
import numpy as np

def subtract_nested_defaultdicts(dict1, dict2):
    """
    Рекурсивно вычитает значения из dict2 из dict1 для всех соответствующих ключей,
    включая вложенные структуры и массивы numpy.
    
    Args:
        dict1: Первый defaultdict
        dict2: Второй defaultdict, значения которого будут вычтены
    
    Returns:
        Новый defaultdict с результатом вычитания
    """
    # Создаем новый defaultdict для результата
    result = defaultdict(list)
    
    # Проходим по всем ключам в первом словаре
    for key in dict1:
        if key in dict2:
            # Если значение - список
            if isinstance(dict1[key], list) and isinstance(dict2[key], list):
                result[key] = []
                # Обрабатываем каждый элемент списка
                for i in range(min(len(dict1[key]), len(dict2[key]))):
                    # Если элемент - массив numpy
                    if isinstance(dict1[key][i], np.ndarray) and isinstance(dict2[key][i], np.ndarray):
                        # Вычитаем массивы
                        result[key].append(dict1[key][i] - dict2[key][i])
                    # Если элемент - вложенный defaultdict
                    elif isinstance(dict1[key][i], defaultdict) and isinstance(dict2[key][i], defaultdict):
                        result[key].append(subtract_nested_defaultdicts(dict1[key][i], dict2[key][i]))
                    # Если элемент - скаляр
                    else:
                        try:
                            result[key].append(dict1[key][i] - dict2[key][i])
                        except (TypeError, ValueError):
                            # Если вычитание невозможно, сохраняем оригинальное значение
                            result[key].append(dict1[key][i])
                
                # Добавляем оставшиеся элементы из первого списка
                if len(dict1[key]) > len(dict2[key]):
                    result[key].extend(dict1[key][len(dict2[key]):])
            
            # Если значение - вложенный defaultdict
            elif isinstance(dict1[key], defaultdict) and isinstance(dict2[key], defaultdict):
                result[key] = subtract_nested_defaultdicts(dict1[key], dict2[key])
            
            # Если значение - массив numpy
            elif isinstance(dict1[key], np.ndarray) and isinstance(dict2[key], np.ndarray):
                result[key] = dict1[key] - dict2[key]
            
            # Если значение - скаляр
            else:
                try:
                    result[key] = dict1[key] - dict2[key]
                except (TypeError, ValueError):
                    # Если вычитание невозможно, сохраняем оригинальное значение
                    result[key] = dict1[key]
        else:
            # Если ключ отсутствует во втором словаре, сохраняем значение из первого
            result[key] = dict1[key]
    
    return result


cow_minus_pig = subtract_nested_defaultdicts(steering_vectors_cow, steering_vectors_pig)

In [ ]:
from generate_image import generate_image
from utilities import compare_images

# Генерируем изображения с разными промптами
prompts = [
    "a pig",
    "a cow",
    "a fish",
    # "Santa Claus",
    # "a bunny and a pig"
]

# Пробуем разные значения alpha (интенсивность добавления стиля)
alphas = [-0.1, -0.2, -0.5, -0.75, -1]

# Для каждого промпта показываем сравнение
for prompt in prompts:
    # Генерируем оригинальное изображение
    original = generate_image(
        model_name=model_name,
        prompt=prompt,
        seed=42,
        not_steer=True
    )

    # Генерируем стилизованные изображения с разными alpha
    styled_images = []
    for alpha in alphas:
        img = generate_image(
            model_name=model_name, 
            prompt=prompt, 
            seed=0, 
            steering_vectors=cow_minus_pig, 
            alpha=alpha, 
            steer_back=True,
            # use_similarity_weight=True
        )
        styled_images.append(img)

    # Отображаем все изображения вместе
    titles = [f"Оригинал: {prompt}"] + [f"alpha={a}" for a in alphas]
    compare_images([original] + styled_images, titles=titles)